This notebook is being used to build the part of the cookbook that will perform the analyses. This will be merged in the future once all parts of the cookbook are complete. 

## Import packages

In [39]:
import pandas as pd
import numpy as np
import scipy.stats
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import re
import seaborn as sns
import statsmodels.stats.multitest
import sys
import urllib3
import json

import CPTAC.Endometrial as CPTAC

Welcome to the CPTAC data service package. Available datasets may be
viewed using CPTAC.list(). In order to access a specific data set,
import a CPTAC subfolder using either 'import CPTAC.Dataset' or 'from
CPTAC import Dataset'.
******
Version: 0.2.5
******
Loading Endometrial CPTAC data:
Loading Dictionary...
Loading Clinical Data...
Loading Acetylation Proteomics Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Mutation Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


## Edit desired variables

In [40]:
'''Mutated gene of interest'''
gene = 'PIK3CA'

'''Database of interest'''
database = 'Endometrial'

'''Select which dataframes you want to analyze '''
proteomics = True
phosphoproteomics = True
transcriptomics = True
sub_type = True

## Get the dataframes

In [42]:
somatic = CPTAC.get_somatic()
proteomics = CPTAC.get_proteomics()
phos = CPTAC.get_phosphoproteomics()

## General gene statistics

In [51]:
'''Get the hotspot3D output '''
hotspot_data = pd.ExcelFile('../HotSpot3D_Output/HotSpot3DOutput.xlsx')

'''Get the sheet name called Clusters to find the protein hotspot'''
clusters = hotspot_data.parse('Clusters')


try:
    '''Check if the gene of interest has a hotspot'''
    hotspot_areas = clusters.loc[clusters['Gene/Drug'] == gene]
    hotspot_mutations = hotspot_areas['Mutation/Gene']
    hotspot_mutations = hotspot_mutations.tolist()

    '''A list of mutations present in the hotspot according to our Hotspot3D output'''
    mutated_hotspot = somatic.loc[(somatic['Location'].isin(hotspot_mutations)) & (somatic['Gene'] == gene)]
    hotspot_patients = mutated_hotspot['Clinical_Patient_Key']
    print('Patients with hotspot mutations: \n')
    print(hotspot_patients)
    
except:
    print('{}' '{}'.format(gene, ' does not have a hotspot in our data.'))
    hotspot = False

Patients with hotspot mutations: 

160      S001
1147     S003
1357     S009
4630     S021
16486    S023
19061    S024
21259    S032
21516    S033
22497    S038
37421    S061
38155    S063
39522    S066
43391    S067
43490    S068
46662    S084
47360    S088
47857    S090
49362    S097
49475    S098
Name: Clinical_Patient_Key, dtype: object
